## 1. Preparation

#### 1.1 Spark preparation

In [0]:
# import necessary libaries
from pyspark.sql.functions import *

In [0]:
# Creating Spark Session
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .appName("ML Model")
         .getOrCreate())

sc = spark.sparkContext

#### 1.2 Prepare a UDF (User Defined Function)

We need to create UDF to embed the ML model we trained in the previous workshop. This model will be used for Posts data sentiment analysis.

In [0]:
# User defined function
def predictions_udf(df, ml_model, stringindexer):
    from pyspark.sql.functions import col, regexp_replace, lower, trim
    from pyspark.ml import PipelineModel

    # Filter out empty body text
    df = df.filter("Body is not null")
    # Making sure the naming of the columns are consistent with the model
    df = df.select(col("Body").alias("text"), col("Tags"))
    # Preprocessing of the feature column
    cleaned = df.withColumn('text', regexp_replace('text', r"http\S+", "")) \
                    .withColumn('text', regexp_replace('text', r"[^a-zA-z]", " ")) \
                    .withColumn('text', regexp_replace('text', r"\s+", " ")) \
                    .withColumn('text', lower('text')) \
                    .withColumn('text', trim('text')) 

    # Load in the saved pipeline model
    model = PipelineModel.load(ml_model)

    # Making the prediction
    prediction = model.transform(df)

    predicted = prediction.select(col('text'), col('Tags'), col('prediction'))

    # Decoding the indexer
    from pyspark.ml.feature import StringIndexerModel, IndexToString

    # Load in the StringIndexer that was saved
    indexer = StringIndexerModel.load(stringindexer)

    # Initialize the IndexToString converter
    i2s = IndexToString(inputCol = 'prediction', outputCol = 'decoded', labels = indexer.labels)
    converted = i2s.transform(predicted)

    # Display the important columns
    return converted

#### 1.3 Load Posts files and ML model

If you can remember our last workshop for machine learning model training, our trained model was saved to `/mnt/deBDProject/model`. Yours name might be different.

In [0]:
display(dbutils.fs.ls("/mnt/deBDProject/model"))

path,name,size,modificationTime
dbfs:/mnt/deBDProject/model/metadata/,metadata/,0,1715538528000
dbfs:/mnt/deBDProject/model/stages/,stages/,0,1715538528000


Let's load the Posts files and the ml model

In [0]:
posts = spark.read.parquet("/mnt/deBDProject/landing/Posts/*")
ml_model = "/mnt/deBDProject/model"
stringindexer = "/mnt/deBDProject/stringindexer"

#### 1.4 Run model to do `Sentiment Analysis`

In [0]:
# # Producing the sentiment analysis
result = predictions_udf(posts,ml_model, stringindexer)
display(result)

text,Tags,prediction,decoded
"Ok, capturing key codes from special symbols produces different results on keyboards with different layouts. But how about the 'common' characters, like a-z? If you have a QWERTY-keyboard, you'd get key code 81 when you type q. When you have an AZERTY-keyboard, do you get code 81 when you press a, since a is where q 'should' be? Or is the mapping done differently? EDIT: The answer I accepted is probably the best solution when you're capturing keys and want to be sure 'a' is really 'a', but as I explain in the comment underneath that, I still am curious how the key codes are 'translated' when using int'l keyboards. That is: sources suggest at least a-z should be consistent, but I cannot find support for this (or someone who actually tried).",,29.0,arrays
To get the api to return JSON change the parameter client=t to client=p,NULL,59.0,hibernate
"Those tags are used by the Pervasive Integration tools. After having contacted Pervasive, I have confirmed these tags are not publicly documented. A potential user must contact Pervasive (http://www.pervasiveintegration.com).",NULL,3.0,jquery
I have an application running Python 2.6.X. Starting the Python interpreter in interactive mode (with or without using virtualenv): readline support working (command history). Starting my application with the same Python interpreter (virtualenv-ed or not): no readline support available when using pdb.set_trace(): cursor keys not working. Operating system is Linux.,,10.0,mysql
"I know there are a lot of questions targeting this issue. Although there are many good naming convention suggestions out there, we had a debate in our team. A team member insist that we should name our table's like that: sProducts and the primary key like that: ProductGuid There are two prefixe's in his system (s, h) so s stands for static and h - sorry i really don't know. I can see absolutely no logic in there. He says that every big oracle and ibm system is working like that. I've never worked with a oracle or ibm system, so is there a convention for adding prefixes like s and h? And what do they stand for? Somebody out there, who does the same? I'm sorry for that question - but I don't want to always add an s and have no clue why...",,209.0,documentation
"I am developing a iPhone photo app in which there is one view, where I am showing all the images taken through that app with the help of uiscrollview. First I simply loaded the scrollview with all the present images. But it made the app slow and also caused the crash giving the following warning: Received memory warning. Level=2 I googled a lot and found that the reason for so may be loading such heavy images all together on a view are resulting in memory warning and crash afterwards. So I opt for the concept of lazy loading. In which I was just loading the three images one at the center(currently shown to user) and the other two on either sides. I did that in following way: - (void)scrollViewDidEndDecelerating:(UIScrollView *)scrollView { pageNum = (int)(scrollView.contentOffset.x / scrollView.frame.size.width); pageNumber = floor(pageNum); [self setImages:pageNumber]; //setting three images } Now there is no crash, no memory warning but the scroll has become quite rough with lots of jerks. Please suggest me how to get rid the roughness or to show all the images same as iPhone 's default photo app. Help is much appreciated. Note: 1. No of images not known, can be unlimited. 2. Xcode 3.2.6 3. ios sdk 4.3 Please consider the link [link]https://www.dropbox.com/sh/h0vwnhhx1acfcb5/W2TQ638udh The link shows a video in which i have shown a photo scroller with lazy loading embedded. Some times it doesn't loads the images so fast also scrolling has become rough.",,6.0,iphone
"So I have a nice persistent allocator class persistent_alloc<T> that allows me to allocate C++ container objects and strings in persistent memory which is backed by an mmaped file that can persist from one run of my program to 

#### 1.5 Summarize which topics are the most popular

In [0]:
# change the column name 
topics = result.withColumnRenamed('decoded', 'topic').select('topic')

# Aggregate the topics and calculate the total qty of each topic
topic_qty = topics.groupBy(col("topic")).agg(count('topic').alias('qty')).orderBy(desc('qty'))
topic_qty.show()

+-----------+---+
|      topic|qty|
+-----------+---+
|         c#|396|
|       java|260|
|  hibernate|155|
| javascript|153|
|     jquery|145|
|        php|118|
|    android| 99|
|        c++| 86|
|     python| 83|
|objective-c| 58|
|      mysql| 51|
|     iphone| 39|
|    asp.net| 38|
|        css| 36|
|       .net| 35|
|        sql| 28|
|       ruby| 21|
|          c| 21|
| sql-server| 19|
|        ios| 19|
+-----------+---+
only showing top 20 rows



#### 1.6 Save the result file to the `BI` folder


Since spark is a distribution system, if you don't anything, the file you saved will be a folder with a couple of files. The files in the folder will be like this.
<img src='https://s3.amazonaws.com/weclouddata/images/data_engineer/ml_prd2.jpg' width='40%'>

In order to save a single file, we need a function to move the csv file move out of the folder, rename it, and delete the folder. leave the single csv file alone. 

In [0]:
# define this function

def crt_sgl_file(result_path):
        # write the result to a folder container several files
        path = "/mnt/deBDProject/BI/ml_result"
        topic_qty.write.option("delimiter", ",").option("header", "true").mode("overwrite").csv(path)

        # list the folder, find the csv file 
        filenames = dbutils.fs.ls(path)
        name = ''
        for filename in filenames:
            if filename.name.endswith('csv'):
                org_name = filename.name

        # copy the csv file to the path you want to save, in this example, we use  "/mnt/deBDProject/BI/ml_result.csv"
        dbutils.fs.cp(path + '/'+ org_name, result_path)

        # delete the folder
        dbutils.fs.rm(path, True)

        print('single file created')

In [0]:
# run the function
result_path = "/mnt/deBDProject/BI/ml_result.csv"

crt_sgl_file(result_path)

single file created
